In [1]:
# First lets improve libraries that we are going to be used in this lab session
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from collections import Counter
import pickle as pkl
import random
import pdb
random.seed(134)

PAD_IDX = 0
UNK_IDX = 1
BATCH_SIZE = 32

In [6]:
import os
os.getcwd()

'/mnt/d/Academic/Nyu/DS1011'

In [2]:
import pandas as pd
train_data = pd.DataFrame.from_csv("/mnt/d/Academic/Nyu/DS1011/hw2_data/snli_train.tsv", sep='\t', index_col = None)
val_data = pd.DataFrame.from_csv("/mnt/d/Academic/Nyu/DS1011/hw2_data/snli_val.tsv", sep='\t', index_col = None)

def convert_to_chars(data):
    return [[t for t in sample.split()] for sample in data]

train_data_1 = convert_to_chars(train_data['sentence1'])
train_data_2 = convert_to_chars(train_data['sentence2'])

val_data_1 = convert_to_chars(val_data['sentence1'])
val_data_2 = convert_to_chars(val_data['sentence2'])

train_label = train_data['label']
val_label = val_data['label']

max_len_1 = max([len(s) for s in train_data_1])
max_len_2 = max([len(s) for s in train_data_2])

val_max_len_1 = max([len(s) for s in val_data_1])
val_max_len_2 = max([len(s) for s in val_data_2])

In [3]:
label_dic = {}
label_dic['contradiction'] = 2
label_dic['entailment'] = 1
label_dic['neutral'] = 0
train_label_vector = [label_dic[i] for i in train_label]
val_label_vector = [label_dic[i] for i in val_label]

In [4]:
ft_home = '/mnt/d/Academic/Nyu/DS1011/'
words_to_load = 100000

import numpy as np

with open(ft_home + 'wiki-news-300d-1M.vec') as f:
    loaded_embeddings_ft = np.zeros((words_to_load, 300))
    token2id_ft = {}
    id2token_ft = {}
    ordered_words_ft = []
    
    token2id_ft['<pad>'] = 0
    id2token_ft[0] = '<pad>'
    token2id_ft['<unk>'] = 1
    id2token_ft[1] = '<unk>' 
    tmp1 = np.zeros((1,300))
    tmp2 = np.random.rand(1,300)
    tmp = np.append(tmp1, tmp2, axis = 0)
    loaded_embeddings_ft = np.append(tmp, loaded_embeddings_ft, axis = 0)
    
    import pdb; pdb.set_trace()
    
    for i, line in enumerate(f):
        #print(i)
        if i >= words_to_load: 
            break
        s = line.split()
        loaded_embeddings_ft[i+2, :] = np.asarray(s[1:])
        token2id_ft[s[0]] = i + 2
        id2token_ft[i+2] = s[0]
        ordered_words_ft.append(s[0])

> <ipython-input-4-a0e0e0ec007d>(23)<module>()
-> for i, line in enumerate(f):
(Pdb) c


In [8]:
# Lets check the dictionary by loading random token from it
import random

random_token_id = random.randint(0, len(id2token_ft)-1)
random_token = id2token_ft[random_token_id]

print ("Token id {} ; token {}".format(random_token_id, id2token_ft[random_token_id]))
print ("Token {}; token id {}".format(random_token, token2id_ft[random_token]))

Token id 95132 ; token pro-gay
Token pro-gay; token id 95132


In [9]:
len(id2token_ft)

100002

In [39]:
class VocabDataset(Dataset):
    """
    Class that represents a train/validation/test dataset that's readable for PyTorch
    Note that this class inherits torch.utils.data.Dataset
    """

    def __init__(self, st1, st2, max_len_1, max_len_2, label, token2id):
        """
        @param data_list: list of character
        @param target_list: list of targets

        """
        self.sentence1 = st1
        self.sentence2 = st2
        self.max_len_1 = max_len_1
        self.max_len_2 = max_len_2
        self.target_list = label
        assert (len(self.sentence1) == len(self.target_list))
        assert (len(self.sentence2) == len(self.target_list))
        self.token2id = token2id

    def __len__(self):
        return len(self.sentence1)

    def __getitem__(self, key):
        """
        Triggered when you call dataset[i]
        """
        token_idx_1 = [self.token2id[c] if c in self.token2id.keys() else self.token2id['<unk>']  for c in self.sentence1[key][:self.max_len_1]]
        token_idx_2 = [self.token2id[c] if c in self.token2id.keys() else self.token2id['<unk>']  for c in self.sentence2[key][:self.max_len_2]]
        label = self.target_list[key]
        return [token_idx_1, len(token_idx_1), token_idx_2, len(token_idx_2), label]

def vocab_collate_func(batch):
    """
    Customized function for DataLoader that dynamically pads the batch so that all
    data have the same length
    """
    data_list_1 = []
    data_list_2 = []
    label_list = []
    length_list_1 = []
    length_list_2 = []
    label_map = {}

    
    for datum in batch:
        label_list.append(datum[4])
        length_list_1.append(datum[1])
        length_list_2.append(datum[3])
    # padding
    
    #import pdb; pdb.set_trace()
    for datum in batch:
        padded_vec_1 = np.pad(np.array(datum[0]),
                                pad_width=((0, max_len_1 - datum[1])),
                                mode="constant", constant_values=0)
        padded_vec_2 = np.pad(np.array(datum[2]),
                                pad_width=((0, max_len_2 - datum[3])),
                                mode="constant", constant_values=0)
        data_list_1.append(padded_vec_1)
        data_list_2.append(padded_vec_2)
    ind_dec_order_1 = np.argsort(length_list_1)[::-1]
    ind_dec_order_2 = np.argsort(length_list_2)[::-1]
    
    reverse_order_1 = []
    reverse_order_2 = []
    #l1_cpy = length_list_1.copy()
    
    for i in range(len(ind_dec_order_1)):
        reverse_order_1.append(ind_dec_order_1.tolist().index(i))
    for i in range(len(ind_dec_order_2)):
        reverse_order_2.append(ind_dec_order_2.tolist().index(i))

    data_list_1 = np.array(data_list_1)[ind_dec_order_1]
    length_list_1 = np.array(length_list_1)[ind_dec_order_1]
    data_list_2 = np.array(data_list_2)[ind_dec_order_2]
    length_list_2 = np.array(length_list_2)[ind_dec_order_2]
    #import pdb; pdb.set_trace()
    return [torch.from_numpy(np.array(data_list_1)), torch.LongTensor(length_list_1), torch.from_numpy(np.array(data_list_2)), torch.LongTensor(length_list_2), torch.LongTensor(label_list), reverse_order_1, reverse_order_2]


In [11]:
# Build train, valid and test dataloaders

train_dataset = VocabDataset(train_data_1, train_data_2, max_len_1, max_len_2, train_label_vector, token2id_ft)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=BATCH_SIZE,
                                           collate_fn=vocab_collate_func,
                                           shuffle=True)

val_dataset = VocabDataset(val_data_1, val_data_2, val_max_len_1, val_max_len_2, val_label_vector, token2id_ft)
val_loader = torch.utils.data.DataLoader(dataset=val_dataset,
                                           batch_size=BATCH_SIZE,
                                           collate_fn=vocab_collate_func,
                                           shuffle=True)

#train_dataset_2 = VocabDataset(train_data_2, token2id_ft)
#train_loader_2 = torch.utils.data.DataLoader(dataset=train_dataset_2,
#                                           batch_size=BATCH_SIZE,
#                                           collate_fn=vocab_collate_func,
#                   shuffle=True)

#val_dataset_1 = VocabDataset(val_data_1, token2id_ft)
#val_loader_1 = torch.utils.data.DataLoader(dataset=val_dataset_1,
#                                           batch_size=BATCH_SIZE,
#                                           collate_fn=vocab_collate_func,
#                                           shuffle=True)

#val_dataset_2 = VocabDataset(val_data_2, token2id_ft)
#val_loader_2 = torch.utils.data.DataLoader(dataset=val_dataset_2,
#                                           batch_size=BATCH_SIZE,
#                                           collate_fn=vocab_collate_func,
#                                           shuffle=True)

In [44]:
class RNN(nn.Module):
    def __init__(self, emb_size, hidden_size, num_layers, num_classes, vocab_size, pt_embed):
        # RNN Accepts the following hyperparams:
        # emb_size: Embedding Size
        # hidden_size: Hidden Size of layer in RNN
        # num_layers: number of layers in RNN
        # num_classes: number of output classes
        # vocab_size: vocabulary size
        super(RNN, self).__init__()
        
        PAD_IDX = 0
        
        super(RNN, self).__init__()

        self.num_layers, self.hidden_size = num_layers, hidden_size
        self.embedding = nn.Embedding(vocab_size, emb_size, padding_idx=PAD_IDX)
        #import pdb; pdb.set_trace()
        #self.embedding.weight.data.copy_(pt_embed)
        self.embedding.from_pretrained(pt_embed, freeze = True)
        
        self.rnn = nn.GRU(emb_size, hidden_size, num_layers, batch_first=True, bidirectional =  True)
        
        self.tf = nn.Sequential(nn.Linear(2*hidden_size, hidden_size), nn.ReLU(inplace=True),
                                nn.Linear(hidden_size,num_classes))
        

    def init_hidden(self, batch_size):
        # Function initializes the activation of recurrent neural net at timestep 0
        # Needs to be in format (num_layers, batch_size, hidden_size)
        hidden = torch.randn(self.num_layers, batch_size, self.hidden_size)

        return hidden

    def forward(self, x_1, lengths_1, x_2, lengths_2, rev_od_1, rev_od_2):
        # reset hidden state

        batch_size, seq_len = x_1.size()
        
        #import pdb; pdb.set_trace()

        #self.hidden_1 = self.init_hidden(batch_size)
        #self.hidden_2 = self.init_hidden(batch_size)
        
        encoded = torch.zeros(0,0)

        # get embedding of characters
        embed_1 = self.embedding(x_1)
        embed_2 = self.embedding(x_2)
        
        # pack padded sequence
        embed_1 = torch.nn.utils.rnn.pack_padded_sequence(embed_1, lengths_1.numpy(), batch_first=True)
        embed_2 = torch.nn.utils.rnn.pack_padded_sequence(embed_2, lengths_2.numpy(), batch_first=True)
        # fprop though RNN
        rnn_out, self.hidden_1 = self.rnn(embed_1)
        rnn_out, self.hidden_2 = self.rnn(embed_2)
        
        #import pdb; pdb.set_trace()
        encoded = torch.cat([torch.index_select(self.hidden_1.permute(1,0,2).sum(dim = 1),0,torch.LongTensor(rev_od_1)), torch.index_select(self.hidden_2.permute(1,0,2).sum(dim = 1),0,torch.LongTensor(rev_od_2))], dim = 1)
        #encoded = torch.mul(torch.index_select(self.hidden_1.permute(1,0,2).sum(dim = 1),0,torch.LongTensor(rev_od_1)), torch.index_select(self.hidden_2.permute(1,0,2).sum(dim = 1),0,torch.LongTensor(rev_od_2)))
        
        logits = self.tf(encoded)
        return logits


In [19]:
#from torch 
def test_model(loader, model):
    """
    Help function that tests the model's performance on a dataset
    @param: loader - data loader for the dataset to test against
    """
    correct = 0
    total = 0
    model.eval()
    for data_1, lengths_1, data_2, lengths_2, labels, rev_od_1, rev_od_2 in loader:
        outputs = F.softmax(model(data_1, lengths_1, data_2, lengths_2, rev_od_1, rev_od_2), dim=1)
        predicted = outputs.max(1, keepdim=True)[1]

        total += labels.size(0)
        correct += predicted.eq(labels.view_as(predicted)).sum().item()
    return (100 * correct / total)


model = RNN(emb_size=300, hidden_size=200, num_layers=1, num_classes=3, vocab_size=len(id2token_ft), pt_embed = torch.FloatTensor(loaded_embeddings_ft))

learning_rate = 0.002
num_epochs = 5 # number epoch to train

# Criterion and Optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
total_step = len(train_loader)

best_acc = 0

for epoch in range(num_epochs):
    for i, (data_1, lengths_1, data_2, lengths_2, labels, rev_od_1, rev_od_2) in enumerate(train_loader):
        model.train()
        optimizer.zero_grad()
        # Forward pass
        #import pdb; pdb.set_trace()
        outputs = model(data_1, lengths_1, data_2, lengths_2, rev_od_1, rev_od_2)
        loss = criterion(outputs, labels)
    
        # Backward and optimize
        loss.backward()
        optimizer.step()

        #print(i)
        
        # validate every 100 iterations
        if i > 0 and i % 100 == 0:
            # validate
            val_acc = test_model(val_loader, model)
            if val_acc >= best_acc:
                best_acc = val_acc
                torch.save(model, "best_rnn_200_mult.pt")
            print('Epoch: [{}/{}], Step: [{}/{}], Validation Acc: {}'.format(
                       epoch+1, num_epochs, i+1, len(train_loader), val_acc))
            print(loss)


/home/clliu/anaconda3/lib/python3.5/site-packages/torch/serialization.py:241: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Epoch: [1/5], Step: [101/3125], Validation Acc: 41.6
tensor(1.0742, grad_fn=<NllLossBackward>)
Epoch: [1/5], Step: [201/3125], Validation Acc: 44.6
tensor(1.0673, grad_fn=<NllLossBackward>)
Epoch: [1/5], Step: [301/3125], Validation Acc: 50.9
tensor(0.9605, grad_fn=<NllLossBackward>)
Epoch: [1/5], Step: [401/3125], Validation Acc: 53.1
tensor(0.9856, grad_fn=<NllLossBackward>)
Epoch: [1/5], Step: [501/3125], Validation Acc: 55.0
tensor(0.7989, grad_fn=<NllLossBackward>)
Epoch: [1/5], Step: [601/3125], Validation Acc: 57.8
tensor(1.0446, grad_fn=<NllLossBackward>)
Epoch: [1/5], Step: [701/3125], Validation Acc: 59.7
tensor(0.8513, grad_fn=<NllLossBackward>)
Epoch: [1/5], Step: [801/3125], Validation Acc: 57.4
tensor(1.0666, grad_fn=<NllLossBackward>)
Epoch: [1/5], Step: [901/3125], Validation Acc: 60.2
tensor(0.9507, grad_fn=<NllLossBackward>)
Epoch: [1/5], Step: [1001/3125], Validation Acc: 59.0
tensor(1.0501, grad_fn=<NllLossBackward>)
Epoch: [1/5], Step: [1101/3125], Validation Acc: 

Epoch: [3/5], Step: [2501/3125], Validation Acc: 67.4
tensor(0.8567, grad_fn=<NllLossBackward>)
Epoch: [3/5], Step: [2601/3125], Validation Acc: 66.7
tensor(0.7262, grad_fn=<NllLossBackward>)
Epoch: [3/5], Step: [2701/3125], Validation Acc: 66.3
tensor(0.8830, grad_fn=<NllLossBackward>)
Epoch: [3/5], Step: [2801/3125], Validation Acc: 66.7
tensor(0.6798, grad_fn=<NllLossBackward>)
Epoch: [3/5], Step: [2901/3125], Validation Acc: 66.8
tensor(0.5617, grad_fn=<NllLossBackward>)
Epoch: [3/5], Step: [3001/3125], Validation Acc: 66.3
tensor(0.7087, grad_fn=<NllLossBackward>)
Epoch: [3/5], Step: [3101/3125], Validation Acc: 67.4
tensor(0.5796, grad_fn=<NllLossBackward>)
Epoch: [4/5], Step: [101/3125], Validation Acc: 67.0
tensor(0.2593, grad_fn=<NllLossBackward>)
Epoch: [4/5], Step: [201/3125], Validation Acc: 66.5
tensor(0.7353, grad_fn=<NllLossBackward>)
Epoch: [4/5], Step: [301/3125], Validation Acc: 67.5
tensor(0.5126, grad_fn=<NllLossBackward>)
Epoch: [4/5], Step: [401/3125], Validation 

In [90]:
class CNN(nn.Module):
    def __init__(self, emb_size, hidden_size, num_layers, num_classes, vocab_size, pt_embed):

        super(CNN, self).__init__()

        self.num_layers, self.hidden_size = num_layers, hidden_size
        self.embedding = nn.Embedding(vocab_size, emb_size, padding_idx=PAD_IDX)
        self.embedding.from_pretrained(pt_embed, freeze = True)
    
        self.conv1 = nn.Conv1d(emb_size, hidden_size, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(hidden_size, hidden_size, kernel_size=3, padding=1)
        self.conv3 = nn.Conv1d(emb_size, hidden_size, kernel_size=3, padding=1)
        self.conv4 = nn.Conv1d(hidden_size, hidden_size, kernel_size=3, padding=1)

        self.tf = nn.Sequential(nn.Linear(2*hidden_size, hidden_size), nn.ReLU(inplace=True),
                                nn.Linear(hidden_size,num_classes))

    def forward(self, x_1, lengths_1, x_2, lengths_2, rev_od_1, rev_od_2):
        data_1 = torch.index_select(x_1, 0, torch.LongTensor(rev_od_1))
        data_2 = torch.index_select(x_2, 0, torch.LongTensor(rev_od_2))
        batch_size, seq_len_1 = data_1.size()
        batch_size, seq_len_2 = data_2.size()
        
        #import pdb; pdb.set_trace()
        embed_1 = self.embedding(data_1)
        hidden_1 = self.conv1(embed_1.transpose(1,2)).transpose(1,2)
        hidden_1 = F.relu(hidden_1.contiguous().view(-1, hidden_1.size(-1))).view(batch_size, seq_len_1, hidden_1.size(-1))

        hidden_1 = self.conv2(hidden_1.transpose(1,2)).transpose(1,2)
        hidden_1 = F.relu(hidden_1.contiguous().view(-1, hidden_1.size(-1))).view(batch_size, seq_len_1, hidden_1.size(-1))
        
        embed_2 = self.embedding(data_2)
        hidden_2 = self.conv3(embed_2.transpose(1,2)).transpose(1,2)
        hidden_2 = F.relu(hidden_2.contiguous().view(-1, hidden_2.size(-1))).view(batch_size, seq_len_2, hidden_2.size(-1))

        hidden_2 = self.conv4(hidden_2.transpose(1,2)).transpose(1,2)
        hidden_2 = F.relu(hidden_2.contiguous().view(-1, hidden_2.size(-1))).view(batch_size, seq_len_2, hidden_2.size(-1))
    
    
        self.maxpool1 = nn.MaxPool1d(seq_len_1)
        self.maxpool2 = nn.MaxPool1d(seq_len_2)
        
        #import pdb; pdb.set_trace()
        hidden_1 = self.maxpool1(hidden_1.permute(0,2,1)).sum(dim = 2)
        hidden_2 = self.maxpool2(hidden_2.permute(0,2,1)).sum(dim = 2)
        encoded = torch.cat([hidden_1, hidden_2], dim = 1)
        
        logits = self.tf(encoded)
        
        return logits

In [ ]:
import copy
def test_model(loader, model):
    """
    Help function that tests the model's performance on a dataset
    @param: loader - data loader for the dataset to test against
    """
    correct = 0
    total = 0
    model.eval()
    for data_1, lengths_1, data_2, lengths_2, labels, rev_od_1, rev_od_2 in loader:
        outputs = F.softmax(model(data_1, lengths_1, data_2, lengths_2, rev_od_1, rev_od_2), dim=1)
        predicted = outputs.max(1, keepdim=True)[1]

        total += labels.size(0)
        correct += predicted.eq(labels.view_as(predicted)).sum().item()
    return (100 * correct / total)


model = CNN(emb_size=300, hidden_size=200, num_layers=1, num_classes=3, vocab_size=len(id2token_ft), pt_embed = torch.FloatTensor(loaded_embeddings_ft))

learning_rate = 0.005
num_epochs = 5 # number epoch to train

# Criterion and Optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
total_step = len(train_loader)
best_acc = 0

for epoch in range(num_epochs):
    for i, (data_1, lengths_1, data_2, lengths_2, labels, rev_od_1, rev_od_2) in enumerate(train_loader):
        model.train()
        optimizer.zero_grad()
        # Forward pass
        #import pdb; pdb.set_trace()
        outputs = model(data_1, lengths_1, data_2, lengths_2, rev_od_1, rev_od_2)
        loss = criterion(outputs, labels)
    
        # Backward and optimize
        loss.backward()
        optimizer.step()

        #print(i)
        
        # validate every 100 iterations
        if i > 0 and i % 50 == 0:
            # validate
            val_acc = test_model(val_loader, model)
            if val_acc >= best_acc:
                best_acc = val_acc
                torch.save(the_model, "best_cnn.pwf")
            print('Epoch: [{}/{}], Step: [{}/{}], Validation Acc: {}'.format(
                       epoch+1, num_epochs, i+1, len(train_loader), val_acc))
            print(loss)


In [25]:
import pandas as pd
train_data = pd.DataFrame.from_csv("/mnt/d/Academic/Nyu/DS1011/hw2_data/mnli_train.tsv", sep='\t', index_col = None)
val_data = pd.DataFrame.from_csv("/mnt/d/Academic/Nyu/DS1011/hw2_data/mnli_val.tsv", sep='\t', index_col = None)

def convert_to_chars(data):
    return [[t for t in sample.split()] for sample in data]

genre = train_data['genre'].unique()

train_data_1 = {}
train_data_2 = {}

val_data_1 = {}
val_data_2 = {}

train_label = {}
val_label = {}

max_len_1 = {}
max_len_2 = {}

val_max_len_1 = {}
val_max_len_2 = {}

label_dic = {}
label_dic['contradiction'] = 2
label_dic['entailment'] = 1
label_dic['neutral'] = 0

train_label_vector = {}
val_label_vector = {}


for i in genre:
    train_data_1[i] = convert_to_chars(train_data[train_data['genre'] == i]['sentence1'])
    train_data_2[i] = convert_to_chars(train_data[train_data['genre'] == i]['sentence2'])
    
    val_data_1[i] = convert_to_chars(val_data[val_data['genre'] == i]['sentence1'])
    val_data_2[i] = convert_to_chars(val_data[val_data['genre'] == i]['sentence2'])
    
    train_label[i] = train_data[train_data['genre'] == i]['label']
    val_label[i] = val_data[val_data['genre'] == i]['label']
    
    max_len_1[i] = max([len(s) for s in train_data_1[i]])
    max_len_2[i] = max([len(s) for s in train_data_2[i]])
    
    train_label_vector[i] = [label_dic[x] for x in train_label[i]]
    val_label_vector[i] = [label_dic[x] for x in val_label[i]]

In [ ]:
# Build train, valid and test 
def test_model(loader, model):
    """
    Help function that tests the model's performance on a dataset
    @param: loader - data loader for the dataset to test against
    """
    correct = 0
    total = 0
    model.eval()
    for data_1, lengths_1, data_2, lengths_2, labels, rev_od_1, rev_od_2 in loader:
        outputs = F.softmax(model(data_1, lengths_1, data_2, lengths_2, rev_od_1, rev_od_2), dim=1)
        predicted = outputs.max(1, keepdim=True)[1]

        total += labels.size(0)
        correct += predicted.eq(labels.view_as(predicted)).sum().item()
    return (100 * correct / total)

for i in genre:
    max_len_1 = max([len(s) for s in val_data_1[i]])
    max_len_2 = max([len(s) for s in val_data_2[i]])
    
    val_dataset = VocabDataset(val_data_1[i], val_data_2[i], val_max_len_1, val_max_len_2, val_label_vector[i], token2id_ft)
    val_loader = torch.utils.data.DataLoader(dataset=val_dataset,
                                           batch_size=BATCH_SIZE,
                                           collate_fn=vocab_collate_func,
                                           shuffle=True)
    best_model = torch.load('best_rnn_2.pt')
    val_acc = test_model(val_loader, best_model)
    print(i)
    print(val_acc)
    
